In [ ]:
%load_ext autoreload
%autoreload 2
import json
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.patches as patches
import numpy as np
from loguru import logger
logger.remove()

from pim.simulator import SimulationExperiment

In [ ]:
T_outbound = 1500
T_inbound = 3000

cheat = False
obstacles = [((-255,50),(-255,200)),((-255,-120),(-255,-20)),((-85,-50),(-85,200)),((-85,-140),(-85,-120))]

parameters = {
    "type": "simulation",
    "T_outbound":T_outbound,
    "T_inbound": T_inbound,
    "min_homing_distance": 300,
    "motor_factor": -0.25 if cheat else 0.25,
    "seed": 1,
    "obstacles": obstacles,
    "cx": {
        "type": "dye",
        "output_layer": "motor",
        "params": {
            "noise": 0.1,
            "pfn_weight_factor": 1,
            "phi": 0.00045,
            "beta": 0.1,
            "k": 0, 
            "epsilon":1e4,
            "length": 1e-3,
            "c_tot": 0.3,
            "cheat": cheat,
            "start_at_stable": False,
            "disable_beta_on_outbound": False
        }
    }
}

parameters = {
    "type": "simulation",
    "T_outbound":T_outbound,
    "T_inbound": T_inbound,
    "min_homing_distance": 300,
    "motor_factor": 1,
    "seed": 1,
    "obstacles": obstacles,
    "cx": {
        "type": "pontine",
        "params": {
            "noise": 0.1,
        }
    }
}

experiment = SimulationExperiment(parameters)
results = experiment.run("test","1")

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.axes()
ax.axis("equal")
results.plot_path(ax)
# ax.scatter([-285.3307936118596,-284.78918968495987],[7.008245766157793,6.8877582412239935])

for line in obstacles:
    ax.add_patch(
        patches.Rectangle(
            line[0],
            1,
            np.abs(line[1][1]-line[0][1]),
            color="gray",
        )
    )

In [ ]:
T_outbound = 1500
T_inbound = T_outbound

path = np.vstack([
    [[T_outbound, np.pi/4]]
])


cheat = True

fig, ax = plt.subplots(2,3, figsize=(15,10))
headings = [np.pi/4, 0, np.pi, np.pi/8, "random", "random_bounded"]
titles = ["Dir of motion","North","South","North-North-East", "Random", "Bounded random"]

for idx, heading in enumerate(headings):
    
    parameters = {
        "type": "simulation",
        "T_outbound":T_outbound,
        "T_inbound": T_inbound,
        "motor_factor": -0.3 if cheat else 0.3,
        "path": path,
        "heading": heading,
        "record": ["memory"],
        "seed": 1,
        "cx": {
            "type": "dye",
            "output_layer": "motor",
            "params": {
                "noise": 0.1,
                "mem_initial": 0.0,
                "pfn_weight_factor": 1,
                "phi": 0.0003,
                "beta": 0.9,
                "k": 0, 
                "epsilon":10,
                "length": 1,
                "c_tot": 0.3,
                "cheat": cheat,
                "start_at_stable": False,
                "holonomic": True,
                "disable_beta_on_outbound": True
            }
        }
    }

#     parameters = {
#         "type": "simulation",
#         "T_outbound":T_outbound,
#         "T_inbound": T_inbound,
#         "motor_factor": 1,
#         "path": path,
# #         "seed": 1,
#         "heading": heading,
# #         "record": ["memory"],
#         "cx": {
#             "type": "pontine",
#             "params": {
#                 "noise": 0.1,
#                 "holonomic": True
#             }
#         }
#     }

    experiment = SimulationExperiment(parameters)
    results = experiment.run("test","1")

    subplot_ax = ax[int(idx/3),idx%3]
    subplot_ax.axis("equal")
    results.plot_path(ax=subplot_ax, decode=False)
    subplot_ax.set_title(titles[idx])